In [231]:
import itertools
import matplotlib.pyplot as plt
import json
import seaborn
from pathlib import Path
import numpy as np
from statistics import mean, median
import pandas as pd

In [232]:
def split_simulations_with_and_without_expansion(simulations):
    with_expansions = []
    without_expansions = []
    for simulations_per_model in simulations:
        for simulations_per_game in simulations_per_model:
            for simulations_per_move in simulations_per_game:
                with_expansions_for_move, without_expansions_for_move = simulations_per_move
                with_expansions.append(with_expansions_for_move)
                without_expansions.append(without_expansions_for_move)
    return with_expansions, without_expansions
            
game = "hex"
results = []
complex_simulations = []
simple_simulations = []
for result_file in Path(f"{game}/complex_rollouts/with_state_evaluator/").iterdir():
    with open(result_file) as f:
        result = json.load(f)
        results.append(result["results"])
        complex_simulations.append(result["complex_simulations"])
        simple_simulations.append(result["simple_simulations"])
results = np.array(results)
wins = results[:, 0:2, :].reshape((20, 4))# + 0.5 * results[:, 1, :]
wins = pd.DataFrame(wins, columns=["first_wins", "second_wins", "first_draws", "second_draws"])
wins = wins[["first_wins", "first_draws", "second_wins", "second_draws"]]
wins.insert(0, "model", np.arange(20))
wins.insert(5, "overall_wins", (wins["first_wins"] + wins["second_wins"]) / 2)
wins.insert(6, "overall_draws", (wins["first_draws"] + wins["second_draws"]) / 2)
wins = wins.set_index("model")
wins

,first_wins,first_draws,second_wins,second_draws,overall_wins,overall_draws
model,,,,,,
0,0.450000,0.0,0.016667,0.0,0.233333,0.0
1,0.500000,0.0,0.033333,0.0,0.266667,0.0
2,0.166667,0.0,0.000000,0.0,0.083333,0.0
3,0.625000,0.0,0.000000,0.0,0.312500,0.0
4,1.000000,0.0,0.000000,0.0,0.500000,0.0
5,0.841667,0.0,0.000000,0.0,0.420833,0.0
6,0.233333,0.0,0.166667,0.0,0.200000,0.0
7,0.633333,0.0,0.316667,0.0,0.475000,0.0
8,0.966667,0.0,0.000000,0.0,0.483333,0.0


In [233]:
index = pd.MultiIndex.from_product([["first", "second", "overall"], ["win", "draw"], ["value", "sem"]])
wins.insert(1, "first_wins_sem", -1)
wins.insert(3, "first_draws_sem", -1)
wins.insert(5, "second_wins_sem", -1)
wins.insert(7, "second_draws_sem", -1)
wins.insert(9, "overall_wins_sem", -1)
wins.insert(11, "overall_draws_sem", -1)
wins.columns = index
for player, type in itertools.product(["first", "second", "overall"], ["win", "draw"]):
    wins[player, type, "sem"] = np.sqrt(wins[player, type, "value"] * (1 - wins[player, type, "value"]) / (240 if player == "overall" else 120))
totals = []
for player, type in itertools.product(["first", "second", "overall"], ["win", "draw"]):
    total = np.mean(wins[player, type, "value"])
    totals.append(total)
    totals.append(np.sqrt(total * (1 - total) / (20 * (240 if player == "overall" else 120))))
wins.loc["Total", :] = totals
wins.index = wins.index.rename("Model")
wins

first                         second                        overall  \
            win            draw            win            draw            win   
          value       sem value  sem     value       sem value  sem     value   
Model                                                                           
0      0.450000  0.045415   0.0  0.0  0.016667  0.011686   0.0  0.0  0.233333   
1      0.500000  0.045644   0.0  0.0  0.033333  0.016387   0.0  0.0  0.266667   
2      0.166667  0.034021   0.0  0.0  0.000000  0.000000   0.0  0.0  0.083333   
3      0.625000  0.044194   0.0  0.0  0.000000  0.000000   0.0  0.0  0.312500   
4      1.000000  0.000000   0.0  0.0  0.000000  0.000000   0.0  0.0  0.500000   
5      0.841667  0.033325   0.0  0.0  0.000000  0.000000   0.0  0.0  0.420833   
6      0.233333  0.038610   0.0  0.0  0.166667  0.034021   0.0  0.0  0.200000   
7      0.633333  0.043991   0.0  0.0  0.316667  0.042465   0.0  0.0  0.475000   
8      0.966667  0.016387   0.0  0.0  0.000000  0.000000   0.0  0.0  0.483333   
9      0.983333  0.011686   0.0  0.0  0.000000  0.000000   0.0  0.0  0.491667   
10     0.008333  0.008299   0.0  0.0  0.500000  0.045644   0.0  0.0  0.254167   
11     0.975000  0.014252   0.0  0.0  0.158333  0.033325   0.0  0.0  0.566667   
12     0.000000  0.000000   0.0  0.0  0.983333  0.011686   0.0  0.0  0.491667   
13     0.675000  0.042757   0.0  0.0  0.000000  0.000000   0.0  0.0  0.337500   
14     0.216667  0.037608   0.0  0.0  0.875000  0.030190   0.0  0.0  0.545833   
15     1.000000  0.000000   0.0  0.0  0.000000  0.000000   0.0  0.0  0.500000   
16     0.000000  0.000000   0.0  0.0  0.308333  0.042157   0.0  0.0  0.154167   
17     0.666667  0.043033   0.0  0.0  0.000000  0.000000   0.0  0.0  0.333333   
18     0.925000  0.024044   0.0  0.0  0.000000  0.000000   0.0  0.0  0.462500   
19     0.933333  0.022771   0.0  0.0  0.975000  0.014252   0.0  0.0  0.954167   
Total  0.590000  0.010040   0.0  0.0  0.216667  0.008409   0.0  0.0  0.403333   

                            
                 draw       
            sem value  sem  
Model                       
0      0.027301   0.0  0.0  
1      0.028545   0.0  0.0  
2      0.017841   0.0  0.0  
3      0.029920   0.0  0.0  
4      0.032275   0.0  0.0  
5      0.031868   0.0  0.0  
6      0.025820   0.0  0.0  
7      0.032234   0.0  0.0  
8      0.032257   0.0  0.0  
9      0.032270   0.0  0.0  
10     0.028104   0.0  0.0  
11     0.031987   0.0  0.0  
12     0.032270   0.0  0.0  
13     0.030523   0.0  0.0  
14     0.032139   0.0  0.0  
15     0.032275   0.0  0.0  
16     0.023309   0.0  0.0  
17     0.030429   0.0  0.0  
18     0.032184   0.0  0.0  
19     0.013499   0.0  0.0  
Total  0.007081   0.0  0.0

In [ ]:
latex_wins = pd.DataFrame(columns=pd.MultiIndex.from_product([["first", "second", "overall"], ["win", "draw"]]))
latex_wins.index = latex_wins.index.rename("Model")
def f(df):
    def inner(player, type):
        return f'${df[player, type, "value"]:.2f} \pm {df[player, type, "sem"] * 1.96:.2f}$'
    return [inner(player, type) for player in ["first", "second", "overall"] for type in ["win", "draw"]]
latex_wins[[("first", "win"), ("first", "draw"), ("second", "win"), ("second", "draw"), ("overall", "win"), ("overall", "draw")]] = wins.apply(func=f, axis="columns", result_type="expand")
# latex_wins = latex_wins.rename(columns={"first": "First player win ratio", "second": "Second player win ratio", "overall": "Overall win ratio", "value": "Value", "sem": "SEM"})
with open(f"{game}_complex_rollouts.txt", "w", encoding="utf-8") as f:
   latex_wins.to_latex(f, escape=False)
with open(f"{game}_complex_rollouts_overall_total.txt", "w", encoding="utf-8") as f:
    latex_wins.loc["Total", "overall"].to_latex(f, escape=False, header=False)
latex_wins

In [ ]:
complex_simulations_with_expansion, complex_simulations_without_expansion = split_simulations_with_and_without_expansion(complex_simulations)
simple_simulations_with_expansion, simple_simulations_without_expansion = split_simulations_with_and_without_expansion(simple_simulations)
average_complex_simulations = mean(complex_simulations_with_expansion), mean(complex_simulations_without_expansion)
average_simple_simulations = mean(simple_simulations_with_expansion), mean(simple_simulations_without_expansion)
print(average_complex_simulations)
print(average_simple_simulations)

In [234]:
complex_simulations_with_expansion, complex_simulations_without_expansion = split_simulations_with_and_without_expansion(complex_simulations)
simple_simulations_with_expansion, simple_simulations_without_expansion = split_simulations_with_and_without_expansion(simple_simulations)
average_complex_simulations = mean(complex_simulations_with_expansion), mean(complex_simulations_without_expansion)
average_simple_simulations = mean(simple_simulations_with_expansion), mean(simple_simulations_without_expansion)
print(average_complex_simulations)
print(average_simple_simulations)

first                            second                   \
                   win             draw              win             draw   
Model                                                                       
0      $0.45 \pm 0.09$  $0.00 \pm 0.00$  $0.02 \pm 0.02$  $0.00 \pm 0.00$   
1      $0.50 \pm 0.09$  $0.00 \pm 0.00$  $0.03 \pm 0.03$  $0.00 \pm 0.00$   
2      $0.17 \pm 0.07$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
3      $0.62 \pm 0.09$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
4      $1.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
5      $0.84 \pm 0.07$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
6      $0.23 \pm 0.08$  $0.00 \pm 0.00$  $0.17 \pm 0.07$  $0.00 \pm 0.00$   
7      $0.63 \pm 0.09$  $0.00 \pm 0.00$  $0.32 \pm 0.08$  $0.00 \pm 0.00$   
8      $0.97 \pm 0.03$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
9      $0.98 \pm 0.02$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
10     $0.01 \pm 0.02$  $0.00 \pm 0.00$  $0.50 \pm 0.09$  $0.00 \pm 0.00$   
11     $0.97 \pm 0.03$  $0.00 \pm 0.00$  $0.16 \pm 0.07$  $0.00 \pm 0.00$   
12     $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.98 \pm 0.02$  $0.00 \pm 0.00$   
13     $0.67 \pm 0.08$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
14     $0.22 \pm 0.07$  $0.00 \pm 0.00$  $0.88 \pm 0.06$  $0.00 \pm 0.00$   
15     $1.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
16     $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.31 \pm 0.08$  $0.00 \pm 0.00$   
17     $0.67 \pm 0.08$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
18     $0.93 \pm 0.05$  $0.00 \pm 0.00$  $0.00 \pm 0.00$  $0.00 \pm 0.00$   
19     $0.93 \pm 0.04$  $0.00 \pm 0.00$  $0.97 \pm 0.03$  $0.00 \pm 0.00$   
Total  $0.59 \pm 0.02$  $0.00 \pm 0.00$  $0.22 \pm 0.02$  $0.00 \pm 0.00$   

               overall                   
                   win             draw  
Model                                    
0      $0.23 \pm 0.05$  $0.00 \pm 0.00$  
1      $0.27 \pm 0.06$  $0.00 \pm 0.00$  
2      $0.08 \pm 0.03$  $0.00 \pm 0.00$  
3      $0.31 \pm 0.06$  $0.00 \pm 0.00$  
4      $0.50 \pm 0.06$  $0.00 \pm 0.00$  
5      $0.42 \pm 0.06$  $0.00 \pm 0.00$  
6      $0.20 \pm 0.05$  $0.00 \pm 0.00$  
7      $0.47 \pm 0.06$  $0.00 \pm 0.00$  
8      $0.48 \pm 0.06$  $0.00 \pm 0.00$  
9      $0.49 \pm 0.06$  $0.00 \pm 0.00$  
10     $0.25 \pm 0.06$  $0.00 \pm 0.00$  
11     $0.57 \pm 0.06$  $0.00 \pm 0.00$  
12     $0.49 \pm 0.06$  $0.00 \pm 0.00$  
13     $0.34 \pm 0.06$  $0.00 \pm 0.00$  
14     $0.55 \pm 0.06$  $0.00 \pm 0.00$  
15     $0.50 \pm 0.06$  $0.00 \pm 0.00$  
16     $0.15 \pm 0.05$  $0.00 \pm 0.00$  
17     $0.33 \pm 0.06$  $0.00 \pm 0.00$  
18     $0.46 \pm 0.06$  $0.00 \pm 0.00$  
19     $0.95 \pm 0.03$  $0.00 \pm 0.00$  
Total  $0.40 \pm 0.01$  $0.00 \pm 0.00$

In [235]:
complex_simulations_with_expansion, complex_simulations_without_expansion = split_simulations_with_and_without_expansion(complex_simulations)
simple_simulations_with_expansion, simple_simulations_without_expansion = split_simulations_with_and_without_expansion(simple_simulations)
average_complex_simulations = mean(complex_simulations_with_expansion), mean(complex_simulations_without_expansion)
average_simple_simulations = mean(simple_simulations_with_expansion), mean(simple_simulations_without_expansion)
print(average_complex_simulations)
print(average_simple_simulations)

(83.94282328011924, 2914.9080395783576)
(156.67311356876994, 2989.506348951933)
